# Data to LED Examples

This notebook will adjust LEDS connected to pins 18-21

In [1]:
print ("Current working directory:")
!pwd
print ("\nAvailable files:")
!ls *.csv

Current working directory:
/home/pi/Desktop/Notebooks

Available files:
raspberrypi-20190715-sensorlog.csv  SAMPLE-rpiz_sensor-1-20180719-sensorlog.csv
raspberrypi-20190716-sensorlog.csv


In [2]:
# Insert the file name
datafile = 'SAMPLE-rpiz_sensor-1-20180719-sensorlog.csv'

In [3]:
# Libraries needed to do the work
import pandas as pd # The Pandas library makes it easier to work with data
import numpy as np # Numpy adds additional numerical methods (math functions)
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

# Read in the csv datafile from above, and save it to the Pandas "dataframe" named sensor_data
# and add the header names to match the data
sensor_data = pd.read_csv(datafile, names=['Timestamp','Light','R','G','B','Temperature','Humidity','A0','A1','A2','A3','A4'])

# Print the first 5 lines of the dataframe named sensor_data
sensor_data.head()

,Timestamp,Light,R,G,B,Temperature,Humidity,A0,A1,A2,A3,A4
0,2018-07-19 13:07:02,4350,(91,85,93),29.958718,98639.521961,146.75,0.372,2.644,0.483,5.105
1,2018-07-19 13:08:02,4341,(91,85,93),29.954461,98635.503785,148.28,0.372,2.647,0.369,5.105
2,2018-07-19 13:09:02,4335,(91,86,93),29.923448,98633.669034,146.89,0.372,2.647,3.713,5.108
3,2018-07-19 13:10:02,4342,(91,85,93),29.980001,98631.545452,146.38,0.372,2.641,0.636,5.099
4,2018-07-19 13:11:02,4341,(91,85,93),29.962063,98629.535126,145.96,0.369,2.647,0.504,5.099


In [11]:
import RPi.GPIO as GPIO   ## Library that allows the GPIO pins to be used
import time
##Setup the GPIO Pins
for pin in range(18,22):
    GPIO.setwarnings(False)  ## Removes any warnings if the pin was previously used
    GPIO.setmode(GPIO.BCM)   ## Allows the use of the GPIO pin numbers instead of the physical pin numbers
    GPIO.setup(pin,GPIO.OUT)
    GPIO.output(pin,GPIO.LOW) ## Sets GPIO pin 21 output voltage low/off

# Lights adjust based on the Humidity readings

More leds will turn on based on the humidity reading found in the data set

In [5]:
max_humidity = sensor_data["Humidity"].max()
print("The maximum humidity in the data is: ", max_humidity)

The maximum humidity in the data is:  98639.52196102617


In [6]:
# Using NumPY Interpolation the below interpolates the sensors data into a given range which in
# this case are the number of LEDS connected
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
int(np.interp(max_humidity, [sensor_data["Humidity"].min(),sensor_data["Humidity"].max()], [18,21]))

21

In [14]:
## Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data["Humidity"]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data["Humidity"].min(),sensor_data["Humidity"].max()], [18,21]))
    #Turn on the number of pins up to the interpolated value
    for pins in range(18, 22):
        GPIO.output(pins,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    for pin_off in range(18,22):
        GPIO.output(pins,GPIO.LOW)
    time.sleep(1)
# Turn all pins off
for pin_off in range(18,22):
        GPIO.output(pin_off,GPIO.LOW)

# Stop Light Colors from Data

Connect:
* Red LED to pin 18
* Yellow LED to pin 19
* Green LED to pin 20

In [16]:
#Using Analog channel 0 (A0) for this example
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
max_a0 = sensor_data["A0"].max()
print("The maximum value for A0 in the data is: ", max_a0)
print(int(np.interp(max_a0, [sensor_data["A0"].min(),sensor_data["A0"].max()], [18,20])))
min_a0 = sensor_data["A0"].min()
print("The min value for A0 in the data is: ", min_a0)
print(int(np.interp(min_a0, [sensor_data["A0"].min(),sensor_data["A0"].max()], [18,20])))
mean_a0 = sensor_data["A0"].mean()
print("The mean value for A0 in the data is: ", mean_a0)
print(int(np.interp(mean_a0, [sensor_data["A0"].min(),sensor_data["A0"].max()], [18,20])))

The maximum value for A0 in the data is:  148.28
20
The min value for A0 in the data is:  145.96
18
The mean value for A0 in the data is:  146.852
18


#### Note: Analog sensors generally have higher readings when nothing is detected and lower reading when something is detected. 

In [17]:
# An easily changeable value to pick which sensor to use
sensor_value_name = "A0"
pins_to_use = [18,20]
# Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data[sensor_value_name]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data[sensor_value_name].min(),sensor_data[sensor_value_name].max()], pins_to_use))
    GPIO.output(pin,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    GPIO.output(pin,GPIO.LOW)
    time.sleep(1)


# Turn all LEDS (pins) Off

In [18]:
for pin in range (18,22):
     GPIO.output(pin,GPIO.LOW)